---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [14]:
def load_energy_file():
    from io import BytesIO
    import requests
    import json
    import pandas as pd
    import numpy as np
    import functools

    # @hidden_cell
    # This function accesses a file in your Object Storage. The definition contains your credentials.
    # You might want to remove those credentials before you share your notebook.
    def get_object_storage_file_with_credentials_f075dc43077a4673a4044b656b793205(container, filename):
        """This functions returns a StringIO object containing
        the file content from Bluemix Object Storage."""

        url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
        data = {}
        headers1 = {'Content-Type': 'application/json'}
        resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
        resp1_body = resp1.json()
        for e1 in resp1_body['token']['catalog']:
            if(e1['type']=='object-store'):
                for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
        s_subject_token = resp1.headers['x-subject-token']
        headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
        resp2 = requests.get(url=url2, headers=headers2)
        # return StringIO(resp2.xxxxx)
        # to be able to work with xls, the following return status is used
        # also update the import statement from StringIO to BytesIO
        return BytesIO(resp2.content)

    # Your data file was loaded into a StringIO object and you can process the data.
    # Please read the documentation of pandas to learn more about your possibilities to load your data.
    # pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html
    data_1 = get_object_storage_file_with_credentials_f075dc43077a4673a4044b656b793205('PythonCourses', 'Energy Indicators.xls')

    #type(data_1)
    xls_file = pd.ExcelFile(data_1) #create instance of ExcelFile class
    
    energy = xls_file.parse('Energy', skiprows=17, skipfooter=38) #read the data from selected Excel sheet
    
    #Drop column 0 and 1 from the energy dataframe
    cols = [0,1]
    energy.drop(energy.columns[cols],axis=1,inplace=True)
    
    #Rename columns
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #Replace ... to NaN
    energy.replace("...", np.NaN, inplace=True)
    
    #Convert Energy Supply from petajoule to gigajoules 
    energy['Energy Supply'] *= 1000000

    return energy


In [12]:
energy = load_energy_file()

#Remove digit from Country
def remove_digit(data):
    import re
    import numpy as np
    
    digit_exist = re.search("\d", data)
    if digit_exist:
        digit_index = digit_exist.start()
        newData = data[:digit_index]
    else:
        digit_index = np.NaN
        newData = data
    return newData
energy['Country'] = energy['Country'].apply(remove_digit)
    

#Remove bracket from Country
def remove_bracket(data):
    import re
    
    bracket_index = data.find('(')
    if bracket_index == -1: #bracket not found
        newData = data
    else:
        bracket_index -= 1
        newData = data[:bracket_index]
    return newData
energy['Country'] = energy['Country'].apply(remove_bracket)

value_list = ['Republic of Korea', 'United States of America', 'United Kingdom of Great Britain and Northern Ireland', 'China, Hong Kong Special Administrative Region', 'Iran']

   
#Replace Country names, define dictionary
di = {"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"}
energy.replace({"Country": di},inplace = True)


value_list = ['South Korea', 'United States', 'United Kingdom', 'Hong Kong', 'Iran']
energy[energy.Country.isin(value_list)]





,Country,Energy Supply,Energy Supply per Capita,% Renewable
43,Hong Kong,5.850000e+08,82.0,0.000000
98,Iran,9.172000e+09,119.0,5.707721
164,South Korea,1.100700e+10,221.0,2.279353
214,United Kingdom,7.920000e+09,124.0,10.600470
216,United States,9.083800e+10,286.0,11.570980


In [15]:
from io import BytesIO
import requests
import json
import pandas as pd
import numpy as np
import functools

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_f075dc43077a4673a4044b656b793205(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""
    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                    if(e2['interface']=='public'and e2['region']=='dallas'):
                        url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return BytesIO(resp2.content)


# Load the world_bank.csv file
GDP = pd.read_csv(get_object_storage_file_with_credentials_f075dc43077a4673a4044b656b793205('PythonCourses', 'world_bank.csv'), skiprows = 4)
GDP.rename(columns={'Country Name': 'Country'}, inplace=True)
#GDP.head()
        
#Replace Country names, define dictionary
dic_2 = { "Korea, Rep.": "South Korea"
        , "Iran, Islamic Rep.": "Iran"
        , "Hong Kong SAR, China": "Hong Kong"}

GDP.replace({'Country': dic_2},inplace = True)

#value_list = ['South Korea', 'Iran', 'Hong Kong']
#GDP[GDP.Country.isin(value_list)]

#GDP

In [16]:
# Load the scimagojr-3.xls file
    
# Your data file was loaded into a StringIO object and you can process the data.
# Please read the documentation of pandas to learn more about your possibilities to load your data.
# pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html
data_2 = get_object_storage_file_with_credentials_f075dc43077a4673a4044b656b793205('PythonCourses', 'scimagojr-3.xlsx')
    
#type(data_2)
xls_file = pd.ExcelFile(data_2) #create instance of ExcelFile class
    
ScimEn = xls_file.parse() #read the data from selected Excel sheet
#ScimEn


In [17]:
# modify datasets
ScimEn_m = ScimEn[:15]
GDP_m = GDP[['Country','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']]



# Join the three datasets
# df = pd.merge(pd.merge(energy, GDP, on='Country'), ScimEn, on='Country')
dfs = [ScimEn_m, energy, GDP_m]
df_final = functools.reduce(lambda left,right: pd.merge(left,right,on='Country'), dfs)
df_final = df_final.set_index('Country')

#df_final


In [18]:
def answer_one():
     return df_final
    
answer_one()    

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [77]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [86]:
dfs = [ScimEn, energy, GDP_m]
df_union = functools.reduce(lambda left,right: pd.merge(left,right,on='Country', how='outer'), dfs)
df_intersect = functools.reduce(lambda left,right: pd.merge(left,right,on='Country'), dfs)
rows_diff = len(df_union) - len(df_intersect)
# rows_diff

156

In [88]:
def answer_two():
    return rows_diff

answer_two()

156

<br>

Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [92]:
def answer_three():
    Top15 = answer_one()
    columns = ['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
    avgGDP = Top15[columns].mean(axis = 1).sort_values(ascending=False)
    return avgGDP

answer_three()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [19]:
def answer_four():
    Top15 = answer_one()
    
    columns = ['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
    Top15["avgGDP"] = Top15[columns].mean(axis = 1)
    Top15_m = Top15.sort_values(by = "avgGDP", ascending = False)
    
    answer = Top15_m
    
    #answer = abs(Top15_m.iloc[5]['2006'] - Top15_m.iloc[5]['2015'])
    return answer

answer_four()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,avgGDP
Country,,,,,,,,,,,,,,,,,,,,,
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.570980,...,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13,1.536434e+13
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,...,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12,6.348609e+12
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,...,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12,5.542208e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,...,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12,3.493025e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,...,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12,2.681725e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.600470,...,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12,2.487907e+12
Brazil,15,8668,8596,60702,14396,7.00,86,1.214900e+10,59.0,69.648030,...,1.957118e+12,2.056809e+12,2.054215e+12,2.208872e+12,2.295245e+12,2.339209e+12,2.409740e+12,2.412231e+12,2.319423e+12,2.189794e+12
Italy,11,10964,10794,111850,26661,10.20,106,6.530000e+09,109.0,33.667230,...,2.234627e+12,2.211154e+12,2.089938e+12,2.125185e+12,2.137439e+12,2.077184e+12,2.040871e+12,2.033868e+12,2.049316e+12,2.120175e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,...,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12,1.769297e+12


### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [110]:
def answer_five():
    Top15 = answer_one()
    answer = Top15["Energy Supply per Capita"].mean()
    return answer

answer_five()

157.59999999999999

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [113]:
def answer_six():
    Top15 = answer_one()
    Top15_m = Top15.sort_values(by = "% Renewable", ascending = False)
    return (Top15_m.iloc[0].name, Top15_m.iloc[0]['% Renewable'])

answer_six()

('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [121]:
def answer_seven():
    Top15 = answer_one()
    Top15["Ratio Citations"] = Top15["Self-citations"] / Top15["Citations"]
    highest_ratio_idx = Top15["Ratio Citations"].idxmax()
    highest_ratio = Top15["Ratio Citations"].max()
    
    return (highest_ratio_idx, highest_ratio)

answer_seven()

('China', 0.68931261793894216)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [124]:
def answer_eight():
    Top15 = answer_one()
    Top15["Est Population"] = Top15["Energy Supply"] / Top15["Energy Supply per Capita"]
    Top15_m = Top15.sort_values(by = "Est Population", ascending = False)
    return Top15_m.iloc[2].name
    
answer_eight()

'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [22]:
def answer_nine():
    
    Top15 = answer_one()
    Top15["Est Population"] = Top15["Energy Supply"] / Top15["Energy Supply per Capita"]
    Top15["Citable docs per person"] = Top15["Citable documents"] / Top15["Est Population"]
    
    # need to convert into float64 
    Top15["Citable docs per person"]=np.float64(Top15["Citable docs per person"])
    Top15["Energy Supply per Capita"]=np.float64(Top15["Energy Supply per Capita"])
      
    answer = Top15.corr()
    #return answer
    return answer.loc['Citable docs per person', 'Energy Supply per Capita']

answer_nine()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,...,2009,2010,2011,2012,2013,2014,2015,avgGDP,Est Population,Citable docs per person
Rank,1.000000,-0.703849,-0.702179,-0.689219,-0.640129,0.335432,-0.589597,-0.656759,-0.253657,0.295288,...,-0.605260,-0.615397,-0.624305,-0.632670,-0.640750,-0.646377,-0.630748,-0.621415,-0.381968,-0.169030
Documents,-0.703849,1.000000,0.999943,0.924086,0.991635,-0.360302,0.617514,0.973529,0.108474,-0.166832,...,0.745377,0.763783,0.784191,0.799996,0.816652,0.830983,0.840711,0.772032,0.602567,-0.056707
Citable documents,-0.702179,0.999943,1.000000,0.920451,0.992488,-0.362623,0.611284,0.974058,0.103424,-0.165754,...,0.739044,0.757710,0.778357,0.794362,0.811247,0.825754,0.835549,0.766023,0.606499,-0.060775
Citations,-0.689219,0.924086,0.920451,1.000000,0.897606,-0.111725,0.854724,0.860706,0.279274,-0.149209,...,0.902614,0.911067,0.921480,0.929063,0.936861,0.944320,0.949564,0.916254,0.425187,0.157238
Self-citations,-0.640129,0.991635,0.992488,0.897606,1.000000,-0.314783,0.580556,0.968527,0.043799,-0.133179,...,0.682508,0.702651,0.725270,0.742572,0.761249,0.777643,0.790658,0.711511,0.643756,-0.087514
Citations per document,0.335432,-0.360302,-0.362623,-0.111725,-0.314783,1.000000,0.306977,-0.467667,0.124916,0.244349,...,-0.141112,-0.156990,-0.170657,-0.182627,-0.193492,-0.199785,-0.254229,-0.160822,-0.348751,0.543795
H index,-0.589597,0.617514,0.611284,0.854724,0.580556,0.306977,1.000000,0.505392,0.399212,-0.063636,...,0.845414,0.842112,0.839667,0.836923,0.834091,0.832627,0.826819,0.843231,0.154473,0.461319
Energy Supply,-0.656759,0.973529,0.974058,0.860706,0.968527,-0.467667,0.505392,1.000000,0.025135,-0.165830,...,0.682087,0.702541,0.725422,0.743307,0.761878,0.778224,0.789133,0.711607,0.721652,-0.214744
Energy Supply per Capita,-0.253657,0.108474,0.103424,0.279274,0.043799,0.124916,0.399212,0.025135,1.000000,-0.093667,...,0.334140,0.320286,0.310022,0.303070,0.292056,0.283620,0.253759,0.319775,-0.472593,0.794001
% Renewable,0.295288,-0.166832,-0.165754,-0.149209,-0.133179,0.244349,-0.063636,-0.165830,-0.093667,1.000000,...,-0.172028,-0.171468,-0.170287,-0.171599,-0.171410,-0.173420,-0.242144,-0.173976,-0.098669,0.055431


In [ ]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [142]:
def answer_ten():
    Top15 = answer_one()
    med = Top15['% Renewable'].median()
    Top15['HighRenew'] = [1 if x >= med else 0 for x in Top15['% Renewable']]
    Top15.sort_values(by = 'Rank', inplace=True)
    return Top15['HighRenew']

answer_ten()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [26]:
def answer_eleven():
    import pandas as pd
    import numpy as np
      
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    Top15 = answer_one()
    
    Top15_Group = pd.DataFrame(columns = ['size', 'sum', 'mean', 'std'])
    
    Top15['Estimated Population'] = (Top15['Energy Supply'] / Top15['Energy Supply per Capita'])
    
    for group, frame in Top15.groupby(ContinentDict):
        Top15_Group.loc[group] = [len(frame), frame['Estimated Population'].sum(),frame['Estimated Population'].mean(),frame['Estimated Population'].std()]
    
    return Top15_Group
    
answer_eleven()

,size,sum,mean,std
Asia,5.0,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1.0,2.331602e+07,2.331602e+07,NaN
Europe,6.0,4.579297e+08,7.632161e+07,3.464767e+07
North America,2.0,3.528552e+08,1.764276e+08,1.996696e+08
South America,1.0,2.059153e+08,2.059153e+08,NaN


In [166]:
def answer_twelve():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    Top15 = Top15.reset_index()
    Top15['Continent'] = [ContinentDict[country] for country in Top15['Country']]
    Top15['bins'] = pd.cut(Top15['% Renewable'],5)   
    Top15_groupby = Top15.groupby(['Continent', 'bins'])
    return Top15_groupby.size()

answer_twelve()

Continent      bins            
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [168]:
def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst'] = (Top15['Energy Supply'] / Top15['Energy Supply per Capita']).astype(float)
    return Top15['PopEst'].apply(lambda x: '{0:,}'.format(x))

answer_thirteen()

Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: PopEst, dtype: object

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!